# This notebook is based on cb_MMT_vels.py
## Important relevant python packages
### note: need the following
#### pip install specutils
#### pip install --upgrade numpy

In [2]:
from time import perf_counter as clock
from os import path

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from astropy.io import fits
import importlib

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.miles_util as lib
##
from astropy.modeling import models
from astropy import units as u
from specutils.spectra import Spectrum1D
from specutils.fitting import fit_lines
import scipy.integrate as integrate
import scipy.constants as const
from matplotlib.patches import Rectangle
from os import listdir
from matplotlib.backends.backend_pdf import PdfPages
from uncertainties import ufloat
#from uncertainties import unumpy as unp
ppcbfits = importlib.import_module('cb_ppxf_tools')

## Define galaxy information

In [3]:
redshifts = {'J0826+4305': 0.6032893, 'J0901+0314': 0.459, 'J0905+5759': 0.711, 'J0944+0930': 0.514,
             'J1107+0417': 0.467, 'J1125-0145': 0.519, 'J1219+0336': 0.451, 'J1232+0723': 0.401,
             'J1341-0321': 0.661, 'J1450+4621': 0.782, 'J1506+5402': 0.608, 'J1558+3957': 0.402,
             'J1613+2834': 0.449, 'J1713+2817': 0.577, 'J2116-0634': 0.728, 'J2118+0017': 0.459,
             'J2140+1209': 0.751}

## Define functions

In [4]:
def closest_position(value,array):
    diff_arr = array - value
    smallest = np.amin(np.abs(diff_arr))
    small_pos = np.where(np.abs(diff_arr) == smallest)

    return small_pos

In [5]:
def calc_vel(lam_or,lam_obs): # lambda in units of Angstroms, return vel in km/s
    del_lam = lam_obs-lam_or
    velocity = del_lam*(1/lam_or)*const.c*(1/1000)

    return velocity

In [6]:
def get_plot_title(filepath):
    fits_file = fits.open(filepath)

    file_char_list = list(filepath)
    spec_pos = filepath.find('spec-')
    title_list = file_char_list[spec_pos:-5]
    title = ''.join(title_list)
    headers = fits_file['PRIMARY'].header
    hh = str(int(headers['PLUG_RA'] * (24/360)))
    if not (len(hh) == 2):
        hh = '0' + hh
    mm = str(int((headers['PLUG_RA'] * (24/360) * 60) % 60))
    if not (len(mm) == 2):
        mm = '0' + mm
    ss = str(np.round((headers['PLUG_RA'] * (24/360) * 60 * 60) % 60, decimals=2))
    if not (len(str(int(np.round((headers['PLUG_RA'] * (24/360) * 60 * 60) % 60)))) == 2):
        ss = '0' + ss
    title = title + '/J' + hh + mm + ss
    return title

## Define the main function

In [7]:
def main(file, instrument):
    hdu = fits.open(file)
    maintable = hdu[1].data

    if instrument == 'MMT':
        name = maintable['SHORT_NAME'][0]
        print('name MMT', name)
    if instrument == 'MaGE':
        name = file[-10:-20:-1][::-1]
        print('name MaGE', name)
    z = redshifts[name]

    mgII = 2799.12
    mgIIl = 2795.5
    mgIIr = 2802.7

    # Only use the wavelength range in common between galaxy and stellar library.
    fluxdata = maintable['FLUX']
    wavelength = maintable['WAVE']
    mask = (wavelength > mgII*(1 + z)-40) & (wavelength < mgII*(1 + z)+40)
    flux = fluxdata[mask]
    galaxy = flux / np.median(flux)  # Normalize spectrum to avoid numerical issues
    wave = wavelength[mask]


    zmg_mask = (wavelength>(mgII*(1 + z)-80)) & (wavelength<(mgII*(1 + z)+80))
    zmg_flux = (fluxdata[zmg_mask] - np.median(fluxdata[zmg_mask]))*(-1)
    zmg_wave = wavelength[zmg_mask]

    mgrndx = np.where(zmg_flux == np.amax(zmg_flux))[0][0]

    spectrum = Spectrum1D(flux=zmg_flux*u.Jy, spectral_axis=zmg_wave*u.AA)

    mgIIl_init = models.Gaussian1D(amplitude=8*u.Jy, mean=zmg_wave[mgrndx]*u.AA, stddev=10*u.AA)
    mgIIr_init = models.Gaussian1D(amplitude=6*u.Jy, mean=zmg_wave[mgrndx-10]*u.AA, stddev=10*u.AA)
    mgl_fit, mgr_fit = fit_lines(spectrum, [mgIIl_init,mgIIr_init], window=10*u.AA)

    std = mgl_fit.stddev/u.AA
    amp = mgl_fit.amplitude/u.Jy
    mean = mgl_fit.mean/u.AA

    yl_fit = mgl_fit(zmg_wave*u.AA)
    yr_fit = mgr_fit(zmg_wave*u.AA)
    gauss_plot = (yr_fit/u.Jy-np.median(fluxdata[zmg_mask]))*(-1)

    square_l = closest_position(zmg_wave[np.where(yl_fit == np.amax(yl_fit))]-(3*std), zmg_wave)[0][0]
    square_r = closest_position(zmg_wave[np.where(yl_fit == np.amax(yl_fit))]+(3*std), zmg_wave)[0][0]


    def get_EW_from_flux(): # recent approach suggested by Alex in blackboard; using 3 sigma to locate absorption
        dlambda = np.array([zmg_wave[x+1]-zmg_wave[x] for x in range(square_l,square_r-1)])
        EqW = np.sum(dlambda *
                     u.AA *
                     (np.median(fluxdata[zmg_mask]) - fluxdata[zmg_mask][square_l:square_r-1]) *
                     (1/np.median(fluxdata[zmg_mask])))

        return EqW


    def get_area_99(): #old method to get equivalent width using area under gaussian
        dlambda = np.array([zmg_wave[x + 1] - zmg_wave[x] for x in range(square_l,square_r-1)])
        gaussian_area = np.sum(dlambda * u.AA * yl_fit[square_l:square_r-1])
        return gaussian_area


    gaussian_area = get_area_99()

    EqW = get_EW_from_flux() # newer equivalent width

    ## Integral method?
    alt_area = integrate.quad(lambda x: amp*(np.e**((-1/2)*(((x-mean)/std)**2))),-np.inf,np.inf)

    # EW = gaussian_area/np.median(fluxdata[zmg_mask]*u.Jy) # old equivalent width calculated with gaussian
    plt.fill_between(zmg_wave*u.AA, 0*u.Jy, np.median(fluxdata[zmg_mask])*u.Jy)


    # get centroid velocity

    centroid_vel = calc_vel(mgII*(1+z), zmg_wave[np.where(yl_fit == np.amax(yl_fit))])
    #print("---Centroid velocity: ", centroid_vel)

    ## Code to find V50 and V90.

    # Determines the area of absorption such that no values above the continuum are included
    # Starting from the calculated center of emission, it returns the spread of the emission line in array index distance.


    def get_abs_spread(data, type):
        cent_indx, cont_median, wave, flux = data[0], data[1], data[2], data[3]
        searching = True
        if type == "blue":
            spread = -1
            while searching:
                if flux[cent_indx + spread] < cont_median:
                    spread -= 1
                if flux[cent_indx + spread] > cont_median:
                    spread += 1
                    searching = False
        if type == "red":
            spread = 1
            while searching:
                if flux[cent_indx + spread] < cont_median:
                    spread += 1
                if flux[cent_indx + spread] > cont_median:
                    spread -= 1
                    searching = False
        return spread


    abs_rspread = get_abs_spread([np.where(yl_fit == np.amax(yl_fit))[0][0],
                                 np.median(fluxdata[zmg_mask]),
                                 zmg_wave,
                                 fluxdata[zmg_mask]],
                                 "red")
    abs_bspread = get_abs_spread([np.where(yl_fit == np.amax(yl_fit))[0][0],
                                 np.median(fluxdata[zmg_mask]),
                                 zmg_wave,
                                 fluxdata[zmg_mask]],
                                 "blue")

    tot_area = EqW * np.median(fluxdata[zmg_mask]) *(1/u.AA) # dimensionless but we know it's angstroms

    #s
    def calc_percentage_pos(data):
        tot_area, percent, cont_median, red_end, wave, flux = data[0], data[1], data[2], data[3], data[4], data[5]
        print(tot_area)
        curr_area = 0
        indx_span = 0
        while curr_area < tot_area*(percent/100):
            indx_span += 1
            dlambda = np.array([wave[x + 1] - wave[x] for x in range(red_end-indx_span, red_end)])
            print("------------------------------ len cut flux: ", len(flux[red_end-indx_span:red_end]),
                  ' ;len dlambda: ', len(dlambda), ' ;red_end-indx_span ', red_end-indx_span)
            curr_area = np.sum(dlambda *
                         (cont_median - flux[red_end-indx_span:red_end]))

        return indx_span, curr_area

    v50_indx_span, v50_area = calc_percentage_pos([tot_area,
                                              50,
                                              np.median(fluxdata[zmg_mask]),
                                              np.where(yl_fit == np.amax(yl_fit))[0][0]+abs_rspread,
                                              zmg_wave,
                                              fluxdata[zmg_mask]])

    v98_indx_span, v98_area = calc_percentage_pos([tot_area,
                                              98,
                                              np.median(fluxdata[zmg_mask]),
                                              np.where(yl_fit == np.amax(yl_fit))[0][0]+abs_rspread,
                                              zmg_wave,
                                              fluxdata[zmg_mask]])

    # Calculate velocity
    red_end = np.where(yl_fit == np.amax(yl_fit))[0][0]+abs_rspread

    v50_wav = zmg_wave[red_end - v50_indx_span]
    v98_wav = zmg_wave[red_end - v98_indx_span]

    v50_vel = (v50_wav - (mgII * (1 + z)))*(1/(mgII * (1 + z)))*const.c*(1/1000)
    v98_vel = (v98_wav - (mgIIl * (1 + z)))*(1/(mgIIl * (1 + z)))*const.c*(1/1000)


    # plotting

    plt.close('all')
    fig = plt.figure()
    ax = fig.add_subplot(1, 2, 1)
    matplotlib.rcParams.update({'font.size': 6})
    fig.suptitle(name)


    rect = Rectangle((zmg_wave[np.where(yl_fit == np.amax(yl_fit))]-(EqW/2)/u.AA, 0), EqW/u.AA,
                     np.median(fluxdata[zmg_mask]), alpha=0.4, color='red', label='EW Rectangle')

    ax.axvspan(zmg_wave[np.where(yl_fit == np.amax(yl_fit))]-(3*std),
               zmg_wave[np.where(yl_fit == np.amax(yl_fit))]+(3*std), alpha=0.3, color='C5', label='Area of emission')

    ax.add_patch(rect)
    ax.grid(True)

    ax.axvline(x=zmg_wave[np.where(yl_fit == np.amax(yl_fit))], color='blue', label='Centroid', alpha=0.4)
    ax.axhline(y=np.median(fluxdata[zmg_mask]), color='cyan', label='Continuum', alpha=0.4)

    #ax.plot(zmg_wave, zmg_flux, color='black', linewidth=0.3)
    # ax.plot(zmg_wave, yl_fit, color='green', linewidth=0.3)
    #ax.plot(zmg_wave, yr_fit, color='orange', linewidth=0.3)
    ax.plot(zmg_wave, gauss_plot, color='m', linewidth=0.3)
    ax.plot(zmg_wave, fluxdata[zmg_mask], color='black', linewidth=0.3)
    ax.set_title("General Information Plot; Centroid Vel: "+ str(centroid_vel) + r"$\frac{km}{s}$")
    ax.tick_params(labelsize=6)
    ax.set_xlabel("$\AA ngstr \ddot{o} ms$")
    ax.set_ylabel("Flux [$10^{-17}$ erg/$cm^{2}$/s/$\AA$]")


    ax.legend()

    ax2 = fig.add_subplot(2, 2, 2)
    ax2.plot(zmg_wave, fluxdata[zmg_mask], color='black', linewidth=0.3)
    ax2.axvline(x=(mgII * (1 + z)), color='red', label='Expected Centroid', alpha=0.4)
    ax2.axvline(x=zmg_wave[np.where(yl_fit == np.amax(yl_fit))], color='green', label='Actual Centroid', alpha=0.4)
    ax2.axhline(y=np.median(fluxdata[zmg_mask]), color='cyan', label='Continuum', alpha=0.4)
    ax2.axvspan(zmg_wave[red_end - v50_indx_span],
               zmg_wave[red_end], alpha=0.3, color='m', label='Area of v50')
    ax2.set_title("v50 Info Plot; $v=$" + str(v50_vel) + r"$\frac{km}{s}$")
    ax2.set_xlabel("$\AA ngstr \ddot{o} ms$")
    ax2.set_ylabel("Flux [$10^{-17}$ erg/$cm^{2}$/s/$\AA$]")


    ax2.legend()
    ax2.grid(True)


    ax3 = fig.add_subplot(2, 2, 4)
    ax3.plot(zmg_wave, fluxdata[zmg_mask], color='black', linewidth=0.3)
    ax3.axvline(x=(mgIIl * (1 + z)), color='blue', label='Expected MgII Blue', alpha=0.4)
    ax3.axvline(x=(mgII * (1 + z)), color='red', label='Expected Centroid', alpha=0.4)
    ax3.axvline(x=zmg_wave[np.where(yl_fit == np.amax(yl_fit))], color='green', label='Emission Centroid', alpha=0.4)
    ax3.axhline(y=np.median(fluxdata[zmg_mask]), color='cyan', label='Continuum', alpha=0.4)
    ax3.axvspan(zmg_wave[red_end - v98_indx_span],
               zmg_wave[red_end], alpha=0.3, color='m', label='Area of v98')
    ax3.set_title("v98 Info Plot; $v=$" + str(v98_vel) + r"$\frac{km}{s}$")
    ax3.set_xlabel("$\AA ngstr \ddot{o} ms$")
    ax3.set_ylabel("Flux [$10^{-17}$ erg/$cm^{2}$/s/$\AA$]")


    ax3.legend()
    ax3.grid(True)

## Define relevant files

In [8]:
MMTgalfol = 'hizea/MMT/'
MMTfiles = [f for f in listdir(MMTgalfol)]
MaGEgalfol = 'hizea/MaGE/'
MaGEfiles = [f for f in listdir(MaGEgalfol)]

# spec-1305-52757-0191.fits is interesting. It might atcually have MgII
# blacklist = ['spec-0326-52375-0382.fits',
#              'spec-0566-52238-0319.fits',
#              'spec-0639-52146-0388.fits',
#              'spec-0986-52443-0580.fits',
#              'spec-1305-52757-0191.fits']

blacklist = ['J1125-0145_MMT.fit']

## Run the code

In [9]:
with PdfPages('V50_V98_MMT.pdf') as pdf:
    for filename in MaGEfiles:
        if filename not in blacklist:
            filepath = MaGEgalfol + filename
            print('\nPlotting MaGE file: ', filename, '..........')
            main(filepath, 'MaGE')
            pdf.savefig()
            plt.close('all')
            print('Success')
    for filename in MMTfiles:
        if filename not in blacklist:
            filepath = MMTgalfol + filename
            print('\nPlotting MMT file: ', filename, '..........')
            main(filepath, 'MMT')
            pdf.savefig()
            plt.close('all')
            print('Success')



print('-----------Finished-----------!')


Plotting MaGE file:  J1232+0723_MAGE.fit ..........
name MaGE J1232+0723
154.48676641204378
------------------------------ len cut flux:  1  ;len dlambda:  1  ;red_end-indx_span  309
------------------------------ len cut flux:  2  ;len dlambda:  2  ;red_end-indx_span  308
------------------------------ len cut flux:  3  ;len dlambda:  3  ;red_end-indx_span  307
------------------------------ len cut flux:  4  ;len dlambda:  4  ;red_end-indx_span  306
------------------------------ len cut flux:  5  ;len dlambda:  5  ;red_end-indx_span  305
------------------------------ len cut flux:  6  ;len dlambda:  6  ;red_end-indx_span  304
------------------------------ len cut flux:  7  ;len dlambda:  7  ;red_end-indx_span  303
------------------------------ len cut flux:  8  ;len dlambda:  8  ;red_end-indx_span  302
------------------------------ len cut flux:  9  ;len dlambda:  9  ;red_end-indx_span  301
------------------------------ len cut flux:  10  ;len dlambda:  10  ;red_end-indx_span 

------------------------------ len cut flux:  47  ;len dlambda:  47  ;red_end-indx_span  263
------------------------------ len cut flux:  48  ;len dlambda:  48  ;red_end-indx_span  262
------------------------------ len cut flux:  49  ;len dlambda:  49  ;red_end-indx_span  261
------------------------------ len cut flux:  50  ;len dlambda:  50  ;red_end-indx_span  260
------------------------------ len cut flux:  51  ;len dlambda:  51  ;red_end-indx_span  259
------------------------------ len cut flux:  52  ;len dlambda:  52  ;red_end-indx_span  258
------------------------------ len cut flux:  53  ;len dlambda:  53  ;red_end-indx_span  257
------------------------------ len cut flux:  54  ;len dlambda:  54  ;red_end-indx_span  256
------------------------------ len cut flux:  55  ;len dlambda:  55  ;red_end-indx_span  255
------------------------------ len cut flux:  56  ;len dlambda:  56  ;red_end-indx_span  254
------------------------------ len cut flux:  57  ;len dlambda:  57  ;

Success

Plotting MaGE file:  J1341-0321_MAGE.fit ..........
name MaGE J1341-0321
200.00674713603834
------------------------------ len cut flux:  1  ;len dlambda:  1  ;red_end-indx_span  251
------------------------------ len cut flux:  2  ;len dlambda:  2  ;red_end-indx_span  250
------------------------------ len cut flux:  3  ;len dlambda:  3  ;red_end-indx_span  249
------------------------------ len cut flux:  4  ;len dlambda:  4  ;red_end-indx_span  248
------------------------------ len cut flux:  5  ;len dlambda:  5  ;red_end-indx_span  247
------------------------------ len cut flux:  6  ;len dlambda:  6  ;red_end-indx_span  246
------------------------------ len cut flux:  7  ;len dlambda:  7  ;red_end-indx_span  245
------------------------------ len cut flux:  8  ;len dlambda:  8  ;red_end-indx_span  244
------------------------------ len cut flux:  9  ;len dlambda:  9  ;red_end-indx_span  243
------------------------------ len cut flux:  10  ;len dlambda:  10  ;red_end-in

------------------------------ len cut flux:  44  ;len dlambda:  44  ;red_end-indx_span  208
------------------------------ len cut flux:  45  ;len dlambda:  45  ;red_end-indx_span  207
------------------------------ len cut flux:  46  ;len dlambda:  46  ;red_end-indx_span  206
------------------------------ len cut flux:  47  ;len dlambda:  47  ;red_end-indx_span  205
------------------------------ len cut flux:  48  ;len dlambda:  48  ;red_end-indx_span  204
------------------------------ len cut flux:  49  ;len dlambda:  49  ;red_end-indx_span  203
------------------------------ len cut flux:  50  ;len dlambda:  50  ;red_end-indx_span  202
------------------------------ len cut flux:  51  ;len dlambda:  51  ;red_end-indx_span  201
------------------------------ len cut flux:  52  ;len dlambda:  52  ;red_end-indx_span  200
------------------------------ len cut flux:  53  ;len dlambda:  53  ;red_end-indx_span  199
------------------------------ len cut flux:  54  ;len dlambda:  54  ;

------------------------------ len cut flux:  4  ;len dlambda:  4  ;red_end-indx_span  243
------------------------------ len cut flux:  5  ;len dlambda:  5  ;red_end-indx_span  242
------------------------------ len cut flux:  6  ;len dlambda:  6  ;red_end-indx_span  241
------------------------------ len cut flux:  7  ;len dlambda:  7  ;red_end-indx_span  240
------------------------------ len cut flux:  8  ;len dlambda:  8  ;red_end-indx_span  239
------------------------------ len cut flux:  9  ;len dlambda:  9  ;red_end-indx_span  238
------------------------------ len cut flux:  10  ;len dlambda:  10  ;red_end-indx_span  237
------------------------------ len cut flux:  11  ;len dlambda:  11  ;red_end-indx_span  236
------------------------------ len cut flux:  12  ;len dlambda:  12  ;red_end-indx_span  235
------------------------------ len cut flux:  13  ;len dlambda:  13  ;red_end-indx_span  234
------------------------------ len cut flux:  14  ;len dlambda:  14  ;red_end-indx

------------------------------ len cut flux:  6  ;len dlambda:  6  ;red_end-indx_span  70
------------------------------ len cut flux:  7  ;len dlambda:  7  ;red_end-indx_span  69
------------------------------ len cut flux:  8  ;len dlambda:  8  ;red_end-indx_span  68
------------------------------ len cut flux:  9  ;len dlambda:  9  ;red_end-indx_span  67
------------------------------ len cut flux:  10  ;len dlambda:  10  ;red_end-indx_span  66
------------------------------ len cut flux:  11  ;len dlambda:  11  ;red_end-indx_span  65
------------------------------ len cut flux:  12  ;len dlambda:  12  ;red_end-indx_span  64
------------------------------ len cut flux:  13  ;len dlambda:  13  ;red_end-indx_span  63
------------------------------ len cut flux:  14  ;len dlambda:  14  ;red_end-indx_span  62
------------------------------ len cut flux:  15  ;len dlambda:  15  ;red_end-indx_span  61
------------------------------ len cut flux:  16  ;len dlambda:  16  ;red_end-indx_span 

------------------------------ len cut flux:  10  ;len dlambda:  10  ;red_end-indx_span  52
------------------------------ len cut flux:  11  ;len dlambda:  11  ;red_end-indx_span  51
------------------------------ len cut flux:  12  ;len dlambda:  12  ;red_end-indx_span  50
------------------------------ len cut flux:  13  ;len dlambda:  13  ;red_end-indx_span  49
------------------------------ len cut flux:  14  ;len dlambda:  14  ;red_end-indx_span  48
------------------------------ len cut flux:  15  ;len dlambda:  15  ;red_end-indx_span  47
------------------------------ len cut flux:  16  ;len dlambda:  16  ;red_end-indx_span  46
------------------------------ len cut flux:  17  ;len dlambda:  17  ;red_end-indx_span  45
------------------------------ len cut flux:  18  ;len dlambda:  18  ;red_end-indx_span  44
------------------------------ len cut flux:  19  ;len dlambda:  19  ;red_end-indx_span  43
------------------------------ len cut flux:  20  ;len dlambda:  20  ;red_end-in

------------------------------ len cut flux:  8  ;len dlambda:  8  ;red_end-indx_span  65
------------------------------ len cut flux:  9  ;len dlambda:  9  ;red_end-indx_span  64
------------------------------ len cut flux:  10  ;len dlambda:  10  ;red_end-indx_span  63
------------------------------ len cut flux:  11  ;len dlambda:  11  ;red_end-indx_span  62
------------------------------ len cut flux:  12  ;len dlambda:  12  ;red_end-indx_span  61
18.954458203648127
------------------------------ len cut flux:  1  ;len dlambda:  1  ;red_end-indx_span  72
------------------------------ len cut flux:  2  ;len dlambda:  2  ;red_end-indx_span  71
------------------------------ len cut flux:  3  ;len dlambda:  3  ;red_end-indx_span  70
------------------------------ len cut flux:  4  ;len dlambda:  4  ;red_end-indx_span  69
------------------------------ len cut flux:  5  ;len dlambda:  5  ;red_end-indx_span  68
------------------------------ len cut flux:  6  ;len dlambda:  6  ;red_end

------------------------------ len cut flux:  15  ;len dlambda:  15  ;red_end-indx_span  55
83.49045817809122
------------------------------ len cut flux:  1  ;len dlambda:  1  ;red_end-indx_span  69
------------------------------ len cut flux:  2  ;len dlambda:  2  ;red_end-indx_span  68
------------------------------ len cut flux:  3  ;len dlambda:  3  ;red_end-indx_span  67
------------------------------ len cut flux:  4  ;len dlambda:  4  ;red_end-indx_span  66
------------------------------ len cut flux:  5  ;len dlambda:  5  ;red_end-indx_span  65
------------------------------ len cut flux:  6  ;len dlambda:  6  ;red_end-indx_span  64
------------------------------ len cut flux:  7  ;len dlambda:  7  ;red_end-indx_span  63
------------------------------ len cut flux:  8  ;len dlambda:  8  ;red_end-indx_span  62
------------------------------ len cut flux:  9  ;len dlambda:  9  ;red_end-indx_span  61
------------------------------ len cut flux:  10  ;len dlambda:  10  ;red_end-in

------------------------------ len cut flux:  6  ;len dlambda:  6  ;red_end-indx_span  56
------------------------------ len cut flux:  7  ;len dlambda:  7  ;red_end-indx_span  55
------------------------------ len cut flux:  8  ;len dlambda:  8  ;red_end-indx_span  54
------------------------------ len cut flux:  9  ;len dlambda:  9  ;red_end-indx_span  53
------------------------------ len cut flux:  10  ;len dlambda:  10  ;red_end-indx_span  52
------------------------------ len cut flux:  11  ;len dlambda:  11  ;red_end-indx_span  51
------------------------------ len cut flux:  12  ;len dlambda:  12  ;red_end-indx_span  50
------------------------------ len cut flux:  13  ;len dlambda:  13  ;red_end-indx_span  49
------------------------------ len cut flux:  14  ;len dlambda:  14  ;red_end-indx_span  48
------------------------------ len cut flux:  15  ;len dlambda:  15  ;red_end-indx_span  47
------------------------------ len cut flux:  16  ;len dlambda:  16  ;red_end-indx_span 